In [1]:
import pandas as pd
import numpy as np
import requests

In [3]:
import requests 

API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
url="https://api.cloudflare.com/client/v4/user/tokens/verify"

headers={
    "Authorization": f"Bearer {API_TOKEN}"
}

response=requests.get(url=url,headers=headers)
# print(response.status_code)
# print(response.json())

In [6]:
l3_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups"
PARAMS = {
    "aggInterval": "1h",
    "dateRange": "1d",
    "format": "json"
}

In [9]:
def fetch_data(url):
    response = requests.get(url, headers=headers, params=PARAMS)
    if response.status_code == 200:
        return response.json().get("result", {}).get("timeseries", [])
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

In [10]:
l3_data = fetch_data(l3_url)
df_l3 = pd.DataFrame(l3_data)


Error 400: {"success":false,"errors":[{"code":7000,"message":"No route for that URI"}],"messages":[],"result":null}


In [11]:
df_l3

""
